## Description

This notebook showcases a **recommended workflow** to prepare the modeling parameters annotated with NeuroCurator for their integration in the **Blue Brain Knowledge Graph** backed by [Blue Brain Nexus](https://bluebrain.github.io/nexus/).

The four main parts of the workflow are:
1. Prepare schemas from Neuroshapes. Publish them in Nexus.
2. Transform source data into JSON-LD / RDF. Register them in Nexus. Recover from errors. Profile invalid data.
3. Create test data for schema validation in Neuroshapes.
4. Clean, _if needed_, created data in Nexus.

For the features of the underlining toolkit, please refer to the [README](https://github.com/BlueBrain/nat/blob/master/kg/README.md) in the directory.

## Getting Started

```
mkdir repos
cd repos

git clone -b prov_literature_annotation --depth=1 https://github.com/INCF/neuroshapes.git
git clone -b master --depth=1 https://github.com/BlueBrain/corpus-thalamus.git
git clone -b master --depth=1 https://github.com/BlueBrain/nat.git

conda create -yn kg python=3.7 jupyter requests
conda activate kg
pip install --extra-index-url https://testpypi.python.org/pypi pyxus==0.5.1

jupyter notebook ./nat/kg/"Literature Annotation - Knowledge Graph.ipynb"
```

## User variables

In [1]:
cd ../..

/Users/fonta/repos


In [2]:
TOKEN = "..."

In [3]:
DEPLOYMENT = "https://bbp-nexus.epfl.ch/staging/v0"

In [4]:
AGENT_IRI_BASE = f"{DEPLOYMENT}/data/.../contributor/agent/v0.1.0"

In [5]:
CLEANING_RULES = [
    (DEPLOYMENT, "{{base}}"),
    ("...", "neurosciencegraph"),
]

In [6]:
NEUROSHAPES_DIR = "./neuroshapes"

In [7]:
ANNOTATION_DIR = "./corpus-thalamus"

In [8]:
UTILS_DIR = "./nat/kg"

## Helpers

In [9]:
%load_ext autoreload

In [10]:
import sys

In [11]:
sys.path.append(UTILS_DIR)

In [12]:
%autoreload 1

In [13]:
%aimport nexus_utils
%aimport kg_utils
%aimport parameters
%aimport annotations

In [14]:
from nexus_utils import *
from kg_utils import *
from parameters import *
from annotations import *

## Nexus client

In [15]:
client = init_client(TOKEN, DEPLOYMENT)

## Configuration

### Commons

In [16]:
commons_config = PipelineConfiguration(TOKEN, DEPLOYMENT, client, "neurosciencegraph", "commons")

### Core

In [17]:
core_config = PipelineConfiguration(TOKEN, DEPLOYMENT, client, "neurosciencegraph", "core")

### Literature Annotation

In [18]:
organization = "literatureannotation"
organization_desc = "Data extracted from the scientific literature."

In [19]:
domain = "modelingparameter"
domain_desc = "Modeling parameters extracted from the scientific literature."

In [20]:
config = PipelineConfiguration(TOKEN, DEPLOYMENT, client, organization, domain, organization_desc, domain_desc)

In [21]:
%%time
pushed_organization = config.create_organization()

<already pushed> literatureannotation
CPU times: user 18.7 ms, sys: 4.95 ms, total: 23.7 ms
Wall time: 323 ms


In [22]:
%%time
pushed_domain = config.create_domain()

<already pushed> modelingparameter
CPU times: user 18.7 ms, sys: 3.71 ms, total: 22.4 ms
Wall time: 110 ms


## Schemas

### Commons

In [23]:
commons_schemas = [
    ("parameter", "v0.1.0"),
    ("variables", "v0.1.0"),
    ("selectors", "v0.1.0"),
    ("annotation", "v0.1.0"),
]

In [24]:
# For demonstration.
commons_config.are_schemas_pushed(commons_schemas)

{'parameter/v0.1.0': True,
 'variables/v0.1.0': True,
 'selectors/v0.1.0': True,
 'annotation/v0.1.0': True}

In [26]:
prepared_commons_schemas = commons_config.load_prepare_schemas(NEUROSHAPES_DIR, commons_schemas)

<prepared> parameter/v0.1.0
<prepared> variables/v0.1.0
<prepared> selectors/v0.1.0
<prepared> annotation/v0.1.0


In [27]:
%%time
pushed_commons_schemas = commons_config.create_publish_schemas(prepared_commons_schemas)

<already pushed> parameter/v0.1.0
<already pushed> variables/v0.1.0
<already pushed> selectors/v0.1.0
<already pushed> annotation/v0.1.0
CPU times: user 78.6 ms, sys: 8.5 ms, total: 87.1 ms
Wall time: 300 ms


In [28]:
# For demonstration.
pushed_commons_schemas

{'parameter/v0.1.0': None,
 'variables/v0.1.0': None,
 'selectors/v0.1.0': None,
 'annotation/v0.1.0': None}

### Core

In [29]:
core_schemas = [
    ("annotation", "v0.1.0"),
    ("selectors", "v0.1.0"),
    ("variables", "v0.1.0"),
]

In [30]:
prepared_core_schemas = core_config.load_prepare_schemas(NEUROSHAPES_DIR, core_schemas)

<prepared> annotation/v0.1.0
<prepared> selectors/v0.1.0
<prepared> variables/v0.1.0


In [31]:
%%time
pushed_core_schemas = core_config.create_publish_schemas(prepared_core_schemas)

<already pushed> annotation/v0.1.0
<already pushed> selectors/v0.1.0
<already pushed> variables/v0.1.0
CPU times: user 57 ms, sys: 7.6 ms, total: 64.6 ms
Wall time: 444 ms


### Literature Annotation

In [32]:
schemas = [
    ("parameterannotation", "v0.1.0"),
    ("pointvalueparameter", "v0.1.0"),
    ("numericaltraceparameter", "v0.1.0"),
    ("functionparameter", "v0.1.0"),
]

In [33]:
prepared_schemas = config.load_prepare_schemas(NEUROSHAPES_DIR, schemas, specific=True)

<prepared> parameterannotation/v0.1.0
<prepared> pointvalueparameter/v0.1.0
<prepared> numericaltraceparameter/v0.1.0
<prepared> functionparameter/v0.1.0


In [34]:
%%time
pushed_schemas = config.create_publish_schemas(prepared_schemas)

<already pushed> parameterannotation/v0.1.0
<already pushed> pointvalueparameter/v0.1.0
<already pushed> numericaltraceparameter/v0.1.0
<already pushed> functionparameter/v0.1.0
CPU times: user 80 ms, sys: 8.06 ms, total: 88.1 ms
Wall time: 337 ms


## Raw annotations

In [35]:
import json
from pathlib import Path
from itertools import chain

In [36]:
# TODO Do it with NAT.
def raw_annotations_from_dir(repo_dir):
    path = Path(repo_dir)
    files = path.glob("*.pcr")
    for x in files:
        if x.stat().st_size > 0:
            with x.open("r", encoding="utf-8") as f:
                json_obj = json.load(f)
                for y in json_obj:
                    yield y

In [37]:
# TODO Do it with NAT.
def raw_annotations_stats(raw_annotations):
    annotated_paper_count = len(set(x["pubId"] for x in raw_annotations))
    print(annotated_paper_count, "papers")  # >= 113

    annotation_count = len(raw_annotations)
    print(annotation_count, "annotations")  # >= 582

    parameter_count = len(list(chain.from_iterable(x["parameters"] for x in raw_annotations)))
    print(parameter_count, "parameters")  # >= 485

In [38]:
unfiltered_raw_annotations = list(raw_annotations_from_dir(ANNOTATION_DIR))

In [39]:
raw_annotations_stats(unfiltered_raw_annotations)
# On 24.09.18:
# 113 papers
# 582 annotations
# 485 parameters

113 papers
582 annotations
485 parameters


### [WIP] Filtered raw annotations

Excluded from the raw annotations:

**Annotations of facts** (i.e. no parameters) because they are currently not modeled in the SHACL shapes.

Annotations with at least a **parameter of type function** because this type of parameter refers to other parameters and handling the IDs in this case is not straightforward with Nexus v0.

In [40]:
def is_selected_raw_annotation(x):
    return x["parameters"] and "function" not in {p["description"]["type"] for p in x["parameters"]}

In [41]:
raw_annotations = [x for x in unfiltered_raw_annotations if is_selected_raw_annotation(x)]

In [42]:
raw_annotations_stats(raw_annotations)
# On 24.09.18:
# 79 papers
# 316 annotations
# 481 parameters

79 papers
316 annotations
481 parameters


## Parameters

### Variable type labels

In [43]:
import csv
import requests

In [44]:
# TODO Use an Ontology Service serving these terms.
def variable_type_labels():
    response = requests.get("https://raw.githubusercontent.com/BlueBrain/nat/master/nat/data/modelingDictionary.csv")
    content = response.content.decode("utf-8")
    reader = csv.reader(content.splitlines(), delimiter=";")
    for x in reader:
        if x and not x[0].startswith("#"):
            yield x[0], x[2]  # type, label

In [45]:
variable_labels_mapping = dict(variable_type_labels())

In [46]:
# For demonstration.
variable_labels_mapping["BBP-121012"]

'interbouton_distance'

### Transformation

In [47]:
parameters = list(transform_parameters(raw_annotations, config.data_context, variable_labels_mapping))

In [48]:
# For demonstration.
print(prepare(parameters[0], CLEANING_RULES))

{
  "@context": "{{base}}/contexts/neurosciencegraph/core/data/v1.0.4",
  "@type": "nsg:PointValueParameter",
  "name": "p-resting_membrane_potential-1dafc5f0",
  "providerId": "1dafc5f0-2f10-11e6-a79a-d0e140998c20",
  "dependentVariable": {
    "@type": "nsg:CompoundNumericalVariable",
    "quantityType": "nsg:BBP-010002",
    "series": [
      {
        "statistic": "mean",
        "unitCode": "mV",
        "value": -70.0
      },
      {
        "statistic": "sem",
        "unitCode": "mV",
        "value": 4.0
      }
    ]
  },
  "requiredTag": [
    {
      "rootId": "NIFCELL:sao1813327414",
      "id": "NIFCELL:sao1813327414",
      "label": "Cell"
    }
  ]
}


### Registration

#### Selection

In [49]:
pv_parameters = list(select_by_type("nsg:PointValueParameter", parameters))
len(pv_parameters)
# On 24.09.18:
# 427

427

In [50]:
nt_parameters = list(select_by_type("nsg:NumericalTraceParameter", parameters))
len(nt_parameters)
# On 24.09.18:
# 54

54

In [51]:
# TODO Upcoming. See filtering section.
# f_parameters = list(select_by_type("nsg:FunctionParameter", parameters))
# len(f_parameters)

#### Push

Invalid parameters - FIXME

In [52]:
def condition_1(x):
    return "series" not in x["dependentVariable"]

def condition_2(x):
    return ("nsg:SimpleNumericalVariable" in x["dependentVariable"]["@type"]
            and "unitCode" not in x["dependentVariable"]["series"])

In [53]:
invalid_pv_parameter_idxs = profiling(pv_parameters, [condition_1, condition_2], flatten=True)

<count> condition_1 15
<count> condition_2 1


In [54]:
# For demonstration.
invalid_pv_parameter_idxs[0]

64

In [55]:
# For demonstration.
config.create_instances("pointvalueparameter", "v0.1.0", pv_parameters, start=invalid_pv_parameter_idxs[0])

<error>
{
  "violations": [
    "Error: Violation Error(<http://www.w3.org/ns/shacl#minCountError>). Node(_:50d62abf0ca585855acb9662bf36131f) MinCount violation. Expected 2, obtained: 0 Node: _:50d62abf0ca585855acb9662bf36131f, Constraint: _:bbff399e9baa42d0df4d8b6c95c34ab1, path: PredicatePath(<https://bbp-nexus.epfl.ch/vocabs/bbp/neurosciencegraph/core/v0.1.0/series>)",
    "Error: Violation Error(<http://www.w3.org/ns/shacl#ShapesFailed>). Node(_:50d62abf0ca585855acb9662bf36131f) Failed property shapes Node: _:50d62abf0ca585855acb9662bf36131f, Constraint: _:f03cd1941eed533b63bfc65ae281d615, path: PredicatePath(<>)"
  ],
  "code": "ShapeConstraintViolations",
  "@context": "https://bbp-nexus.epfl.ch/staging/v0/contexts/nexus/core/error/v0.1.0"
}
<data>
{
  "@context": "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/data/v1.0.4",
  "@type": "nsg:PointValueParameter",
  "name": "p-connectivity-274c8102",
  "providerId": "274c8102-ffde-11e5-a94f-c869cd917532",
  "d

Point value parameters

In [56]:
len(pv_parameters) - len(invalid_pv_parameter_idxs)

411

In [57]:
%%time
config.create_instances("pointvalueparameter", "v0.1.0", pv_parameters, exclude_idxs=invalid_pv_parameter_idxs)

<count> 411
CPU times: user 15.8 s, sys: 1.13 s, total: 16.9 s
Wall time: 5min 29s


Numerical trace parameters

In [58]:
len(nt_parameters)

54

In [59]:
%%time
config.create_instances("numericaltraceparameter", "v0.1.0", nt_parameters)

<count> 54
CPU times: user 2.17 s, sys: 156 ms, total: 2.33 s
Wall time: 42.8 s


Function parameters

In [60]:
# TODO Upcoming. See filtering section.
# len(f_parameters)

In [61]:
# TODO Upcoming. See filtering section.
# %%time
# config.create_instances("functionparameter", "v0.1.0", f_parameters)

## [Nexus v0] Parameter IDs mapping

In [62]:
# For demonstration.
config.instances_by_schema("pointvalueparameter", "v0.1.0")

<count> 411
<warning> More than one result page. Iteration required.


In [63]:
# For demonstration.
config.instances_by_schema("pointvalueparameter", "v0.1.0", resolved=True).results[0]

<count> 411
<warning> More than one result page. Iteration required.


In [64]:
len(parameters) - len(invalid_pv_parameter_idxs)

465

In [65]:
# Note: Need to wait a bit so that Nexus makes the instances available for retrieve_all_results().
import time
time.sleep(30)

In [66]:
pushed_parameters_search = config.instances_of_domain(resolved=True)

<count> 465
<warning> More than one result page. Iteration required.


In [67]:
pushed_parameters = config.retrieve_all_results(pushed_parameters_search)

In [68]:
parameter_uuid_mapping = dict(uuid_iri_mapping(pushed_parameters))

## Annotations

### Transformation

Invalid annotations because of invalid parameters - FIXME

In [69]:
invalid_pv_parameter_uuids = {pv_parameters[i]["providerId"] for i in invalid_pv_parameter_idxs}

In [70]:
valid_raw_annotations = [x for x in raw_annotations
                         if not {y["id"] for y in x["parameters"]} & invalid_pv_parameter_uuids]

In [71]:
len(raw_annotations) - len(valid_raw_annotations)

10

Normal flow

In [72]:
annotations = list(transform_annotations(valid_raw_annotations, config.data_context, AGENT_IRI_BASE,
                                         parameter_uuid_mapping))

In [73]:
# For demonstration.
print(prepare(annotations[-1], CLEANING_RULES))

{
  "@context": "{{base}}/contexts/neurosciencegraph/core/data/v1.0.4",
  "@type": "nsg:ParameterAnnotation",
  "name": "a-figure-5bec10d2",
  "providerId": "5bec10d2-a1b4-11e6-8ba4-64006a4c56ef",
  "contribution": {
    "agent": [
      {
        "@id": "{{base}}/data/neurosciencegraph/contributor/agent/v0.1.0/124d1066-41cd-4d1b-b0ad-eed2672cf927",
        "@type": "prov:Agent"
      }
    ]
  },
  "hasTarget": {
    "@type": "nsg:FigureTarget",
    "hasSource": "https://www.ncbi.nlm.nih.gov/pubmed/12509484",
    "hasSelector": {
      "@type": "nsg:FigureSelector",
      "index": "3.D"
    }
  },
  "hasBody": [
    {
      "@id": "{{base}}/data/literatureannotation/modelingparameter/numericaltraceparameter/v0.1.0/b97c843c-1613-4a51-920d-b6cfa8525028",
      "@type": "nsg:Parameter"
    }
  ],
  "keywords": [
    {
      "@id": "NIFORG:birnlex_254",
      "label": "Wistar Rat"
    }
  ]
}


### Registration

#### Push

In [74]:
len(annotations)

306

In [75]:
%%time
config.create_instances("parameterannotation", "v0.1.0", annotations)

<count> 306
CPU times: user 11.8 s, sys: 827 ms, total: 12.6 s
Wall time: 4min 40s


In [76]:
# For demonstration.
config.create_instances("parameterannotation", "v0.0.0", annotations)

<error> Schema does not exist!
<index> 0
<count> 0


---

## Neuroshapes test data generation

This part of the workflow is intended to be done **before pushing the schemas and the data**.

In [77]:
CLEANING_RULES.append((f"{organization}/{domain}", f"neurosciencegraph/{organization}"))

In [78]:
tdconfig = TestDataConfiguration(organization, NEUROSHAPES_DIR, CLEANING_RULES)

### Valid data

#### ParameterAnnotation

In [85]:
# To find interesting examples.
# prettify([x for x in annotations if "TextPositionTarget" in x["hasTarget"]["@type"]][0])

In [86]:
pa_schema_name = "parameterannotation"
pa_schema_version = "v0.1.0"
pa_optional_keys = ["keywords", "comment", "experimentalProperties"]

TextPositionTarget

In [87]:
tdconfig.write(
    find("e6914b74-ed2a-11e5-b291-3417ebb8f5ca", annotations),
    pa_schema_name, pa_schema_version,
    pa_optional_keys,
    flavour="text")

<written> parameterannotation/v0.1.0/auto-all-fields-text.json
<written> parameterannotation/v0.1.0/auto-min-fields-text.json


FigureTarget

In [88]:
tdconfig.write(
    find("1132613e-31b1-11e8-b594-64006a67e5d0", annotations),
    pa_schema_name, pa_schema_version,
    pa_optional_keys,
    flavour="figure")

<written> parameterannotation/v0.1.0/auto-all-fields-figure.json
<written> parameterannotation/v0.1.0/auto-min-fields-figure.json


EquationTarget

In [89]:
tdconfig.write(
    find("c3af1a82-c5a7-11e5-b8a7-3417ebb8f5ca", annotations),
    pa_schema_name, pa_schema_version,
    pa_optional_keys,
    flavour="equation")

<written> parameterannotation/v0.1.0/auto-all-fields-equation.json
<written> parameterannotation/v0.1.0/auto-min-fields-equation.json


TableTarget

In [90]:
tdconfig.write(
    find("5ffe9b02-ecf3-11e5-b708-64006a4c56ef", annotations),
    pa_schema_name, pa_schema_version,
    pa_optional_keys,
    flavour="table")

<written> parameterannotation/v0.1.0/auto-all-fields-table.json
<written> parameterannotation/v0.1.0/auto-min-fields-table.json


AreaTarget

In [91]:
tdconfig.write(
    find("52afbf40-04bc-11e6-b795-64006a4c56ef", annotations),
    pa_schema_name, pa_schema_version,
    pa_optional_keys,
    flavour="area")

<written> parameterannotation/v0.1.0/auto-all-fields-area.json
<written> parameterannotation/v0.1.0/auto-min-fields-area.json


#### PointValueParameter

In [92]:
pvp_schema_name = "pointvalueparameter"
pvp_schema_version = "v0.1.0"
pvp_optional_keys = ["requiredTag"]

SimpleNumericalVariable

In [93]:
tdconfig.write(
    find("870f0288-061b-11e6-9d1c-c869cd917532", parameters),
    pvp_schema_name, pvp_schema_version,
    pvp_optional_keys,
    flavour="simple")

<written> pointvalueparameter/v0.1.0/auto-all-fields-simple.json
<written> pointvalueparameter/v0.1.0/auto-min-fields-simple.json


CompoundNumericalVariable


In [94]:
tdconfig.write(
    find("7516d66e-0bb4-11e6-843d-64006a4c56ef", parameters),
    pvp_schema_name, pvp_schema_version,
    pvp_optional_keys,
    flavour="compound")

<written> pointvalueparameter/v0.1.0/auto-all-fields-compound.json
<written> pointvalueparameter/v0.1.0/auto-min-fields-compound.json


#### NumericalTraceParameter

In [95]:
nt_schema_name = "numericaltraceparameter"
nt_schema_version = "v0.1.0"
nt_optional_keys = ["requiredTag"]

SimpleNumericalVariable - SimpleNumericalVariable

In [96]:
tdconfig.write(
    find("0ebdc338-6d38-11e6-b432-64006a4c56ef", parameters),
    nt_schema_name, nt_schema_version,
    nt_optional_keys,
    flavour="simple-simple")

<written> numericaltraceparameter/v0.1.0/auto-all-fields-simple-simple.json
<written> numericaltraceparameter/v0.1.0/auto-min-fields-simple-simple.json


SimpleNumericalVariable - CompoundNumericalVariable

In [97]:
len([x for x in parameters
     if "NumericalTraceParameter" in x["@type"]
     and "SimpleNumericalVariable" in x["dependentVariable"]["@type"]
     and "CompoundNumericalVariable" in {y["@type"] for y in x["independentVariable"]}])
# On 24.09.18:
# 0, no data

0

CompoundNumericalVariable - SimpleNumericalVariable

In [98]:
tdconfig.write(
    find("db0e39fe-9a9c-11e6-974a-64006a4c56ef", parameters),
    nt_schema_name, nt_schema_version,
    nt_optional_keys,
    flavour="compound-simple")

<written> numericaltraceparameter/v0.1.0/auto-all-fields-compound-simple.json
<written> numericaltraceparameter/v0.1.0/auto-min-fields-compound-simple.json


CompoundNumericalVariable - CompoundNumericalVariable

In [99]:
len([x for x in parameters
     if "NumericalTraceParameter" in x["@type"]
     and "CompoundNumericalVariable" in x["dependentVariable"]["@type"]
     and "CompoundNumericalVariable" in {y["@type"] for y in x["independentVariable"]}])
# On 24.09.18:
# 0, no data

0

#### FunctionParameter

In [100]:
f_schema_name = "functionparameter"
f_schema_version = "v0.1.0"
f_optional_keys = ["equationParameters", "requiredTag"]

In [101]:
# TODO Upcoming. See filtering section.

# Meanwhile, ad hoc and temporary code to generate the test data for Neuroshapes.

fp = find("71233092-eb80-11e5-a9b7-64006a4c56ef",
          chain.from_iterable(x["parameters"] for x in unfiltered_raw_annotations),
          "id")

tfp = transform_parameter(fp, config.data_context, variable_labels_mapping)

for i, x in enumerate(tfp["equationParameters"]):
    puuid = x["@id"]
    pbase = "{{base}}/data/neurosciencegraph/literatureannotation/pointvalueparameter/v0.1.0"
    tfp["equationParameters"][i]["@id"] =  f"{pbase}/{puuid}"

tdconfig.write(
    tfp,  # find("71233092-eb80-11e5-a9b7-64006a4c56ef", parameters),
    f_schema_name, f_schema_version,
    f_optional_keys)

<written> functionparameter/v0.1.0/auto-all-fields.json
<written> functionparameter/v0.1.0/auto-min-fields.json


### Invalid data

#### ParameterAnnotation

TextPositionTarget

In [102]:
tdconfig.write_missing(
    pa_schema_name, pa_schema_version, "auto-min-fields-text",
    [
        ("name", "entityshape-name"),
        ("contribution", "contribution"),
        ("contribution.agent", "contributionshape-agent"),
        ("hasTarget", "hastarget"),
        ("hasBody", "hasbody"),
        ("hasTarget.hasSource", "selectortargetshape-hassource"),
        # hasTarget.hasSelector: Same for the other specialized TargetShape.
        ("hasTarget.hasSelector", "textpositiontargetshape-hasselector"),
        ("hasTarget.hasSelector.start", "textpositionselectorshape-start"),
        ("hasTarget.hasSelector.end", "textpositionselectorshape-end"),
    ])

<written> parameterannotation/v0.1.0/auto-missing-entityshape-name.json
<written> parameterannotation/v0.1.0/auto-missing-contribution.json
<written> parameterannotation/v0.1.0/auto-missing-contributionshape-agent.json
<written> parameterannotation/v0.1.0/auto-missing-hastarget.json
<written> parameterannotation/v0.1.0/auto-missing-hasbody.json
<written> parameterannotation/v0.1.0/auto-missing-selectortargetshape-hassource.json
<written> parameterannotation/v0.1.0/auto-missing-textpositiontargetshape-hasselector.json
<written> parameterannotation/v0.1.0/auto-missing-textpositionselectorshape-start.json
<written> parameterannotation/v0.1.0/auto-missing-textpositionselectorshape-end.json


FigureTarget

In [103]:
tdconfig.write_missing(
    pa_schema_name, pa_schema_version, "auto-min-fields-figure",
    [
        # hasTarget.hasSelector.index: Same for the other specialized IndexSelectorShape (Equation and Table).
        ("hasTarget.hasSelector.index", "indexselectorshape-index"),
    ])

<written> parameterannotation/v0.1.0/auto-missing-indexselectorshape-index.json


AreaTarget

In [104]:
tdconfig.write_missing(
    pa_schema_name, pa_schema_version, "auto-min-fields-area",
    [
        ("hasTarget.hasSelector.rdf:value", "fragmentselectorshape-value"),
        ("hasTarget.hasSelector.dcterms:conformsTo", "fragmentselectorshape-conformsto"),
    ])

<written> parameterannotation/v0.1.0/auto-missing-fragmentselectorshape-value.json
<written> parameterannotation/v0.1.0/auto-missing-fragmentselectorshape-conformsto.json


#### PointValueParameter

In [105]:
tdconfig.write_missing(
    pvp_schema_name, pvp_schema_version, "auto-min-fields-simple",
    [
        ("dependentVariable.series", "series-simple"),
        ("dependentVariable.quantityType", "variableshape-quantitytype-simple"),
        ("dependentVariable.series.value", "valueshape-value-simple"),
        ("dependentVariable.series.unitCode", "algebraicvalueshape-unitcode-simple"),
    ])

<written> pointvalueparameter/v0.1.0/auto-missing-series-simple.json
<written> pointvalueparameter/v0.1.0/auto-missing-variableshape-quantitytype-simple.json
<written> pointvalueparameter/v0.1.0/auto-missing-valueshape-value-simple.json
<written> pointvalueparameter/v0.1.0/auto-missing-algebraicvalueshape-unitcode-simple.json


In [106]:
tdconfig.write_missing(
    pvp_schema_name, pvp_schema_version, "auto-min-fields-compound",
    [
        ("dependentVariable.series", "series-compound"),
        ("dependentVariable.quantityType", "variableshape-quantitytype-compound"),
    ])

<written> pointvalueparameter/v0.1.0/auto-missing-series-compound.json
<written> pointvalueparameter/v0.1.0/auto-missing-variableshape-quantitytype-compound.json


#### NumericalTraceParameter

In [107]:
tdconfig.write_missing(
    nt_schema_name, nt_schema_version, "auto-min-fields-simple-simple",
    [
        ("independentVariable", "indpendentvariable-simple"),
    ])

<written> numericaltraceparameter/v0.1.0/auto-missing-indpendentvariable-simple.json


#### FunctionParameter

In [108]:
tdconfig.write_missing(
    f_schema_name, f_schema_version, "auto-min-fields",
    [
        ("equation", "equation"),
    ])

<written> functionparameter/v0.1.0/auto-missing-equation.json


---

## Cleaning - KNOW WHAT YOU ARE DOING

In [109]:
# %%time
# config.clean(organization=False, domain=False)